In [5]:
# Setup - Run this cell first
import polars as pl

# Load data
airlines = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airlines.csv')
airports = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airports.csv')
flights = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_flights.csv', null_values='NA')
planes = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_planes.csv', null_values='NA')
weather = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_weather.csv', null_values='NA', schema_overrides={'precip': pl.Float64, 'visib': pl.Float64})

flights = flights.with_columns(pl.col("time_hour").str.strptime(pl.Datetime))
weather = weather.with_columns(pl.col("time_hour").str.strptime(pl.Datetime))

# Create SQL context
ctx = pl.SQLContext(
    airlines=airlines,
    airports=airports,
    flights=flights,
    planes=planes,
    weather=weather,
    eager_execution=True
)

print("Setup complete! Tables available:")
print(ctx.execute("SHOW TABLES"))

Setup complete! Tables available:
shape: (5, 1)
┌──────────┐
│ name     │
│ ---      │
│ str      │
╞══════════╡
│ airlines │
│ airports │
│ flights  │
│ planes   │
│ weather  │
└──────────┘


/tmp/ipython-input-745455298.py:15: DeprecationWarning: The argument `eager_execution` for `SQLContext.__init__` is deprecated. It has been renamed to `eager`.
  ctx = pl.SQLContext(


In [8]:
# Question 1.1

result = ctx.execute("""
SELECT DISTINCT carrier
FROM airlines;
""")

print(result)


shape: (16, 1)
┌─────────┐
│ carrier │
│ ---     │
│ str     │
╞═════════╡
│ 9E      │
│ AA      │
│ AS      │
│ B6      │
│ DL      │
│ …       │
│ UA      │
│ US      │
│ VX      │
│ WN      │
│ YV      │
└─────────┘


In [9]:
# Question 1.2

result = ctx.execute("""
SELECT dest, COUNT(*) AS flight_count
FROM flights
GROUP BY dest
ORDER BY flight_count DESC
LIMIT 10;
""")

print(result)

shape: (10, 2)
┌──────┬──────────────┐
│ dest ┆ flight_count │
│ ---  ┆ ---          │
│ str  ┆ u32          │
╞══════╪══════════════╡
│ ORD  ┆ 17283        │
│ ATL  ┆ 17215        │
│ LAX  ┆ 16174        │
│ BOS  ┆ 15508        │
│ MCO  ┆ 14082        │
│ CLT  ┆ 14064        │
│ SFO  ┆ 13331        │
│ FLL  ┆ 12055        │
│ MIA  ┆ 11728        │
│ DCA  ┆ 9705         │
└──────┴──────────────┘


In [10]:
# Question 1.3

result = ctx.execute("""
SELECT *
FROM flights
WHERE dep_delay > 120;
""")

print(result)

shape: (9_723, 19)
┌──────┬───────┬─────┬──────────┬───┬──────────┬──────┬────────┬─────────────────────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ distance ┆ hour ┆ minute ┆ time_hour               │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---      ┆ ---  ┆ ---    ┆ ---                     │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ i64      ┆ i64  ┆ i64    ┆ datetime[μs, UTC]       │
╞══════╪═══════╪═════╪══════════╪═══╪══════════╪══════╪════════╪═════════════════════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 848      ┆ … ┆ 184      ┆ 18   ┆ 35     ┆ 2013-01-01 23:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 957      ┆ … ┆ 200      ┆ 7    ┆ 33     ┆ 2013-01-01 12:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1114     ┆ … ┆ 1416     ┆ 9    ┆ 0      ┆ 2013-01-01 14:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1540     ┆ … ┆ 1598     ┆ 13   ┆ 38     ┆ 2013-01-01 18:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1815     ┆ … ┆ 1134     ┆ 13   ┆ 25     ┆ 2013-01-01 18:00:00 UTC │
│ …    ┆ …     ┆ …   ┆ …        ┆ … ┆ …        ┆ …    ┆ …      ┆ …     

In [11]:
# Question 2.1

result = ctx.execute("""
SELECT origin, AVG(dep_delay) AS avg_dep_delay
FROM flights
GROUP BY origin;
""")

print(result)

shape: (3, 2)
┌────────┬───────────────┐
│ origin ┆ avg_dep_delay │
│ ---    ┆ ---           │
│ str    ┆ f64           │
╞════════╪═══════════════╡
│ LGA    ┆ 10.346876     │
│ EWR    ┆ 15.107954     │
│ JFK    ┆ 12.112159     │
└────────┴───────────────┘


In [12]:
# Question 2.2

result = ctx.execute("""
SELECT month, COUNT(*) AS flight_count
FROM flights
GROUP BY month
ORDER BY flight_count DESC
LIMIT 1;
""")

print(result)

shape: (1, 2)
┌───────┬──────────────┐
│ month ┆ flight_count │
│ ---   ┆ ---          │
│ i64   ┆ u32          │
╞═══════╪══════════════╡
│ 7     ┆ 29425        │
└───────┴──────────────┘


In [13]:
# Question 2.3

result = ctx.execute("""
SELECT
  carrier,
  100.0 * SUM(CASE WHEN dep_delay <= 15 THEN 1 ELSE 0 END) / COUNT(*) AS on_time_rate
FROM flights
GROUP BY carrier
ORDER BY on_time_rate DESC;
""")

print(result)

shape: (16, 2)
┌─────────┬──────────────┐
│ carrier ┆ on_time_rate │
│ ---     ┆ ---          │
│ str     ┆ f64          │
╞═════════╪══════════════╡
│ HA      ┆ 92.982456    │
│ AS      ┆ 86.554622    │
│ US      ┆ 84.987339    │
│ DL      ┆ 83.074205    │
│ AA      ┆ 82.437594    │
│ …       ┆ …            │
│ FL      ┆ 71.687117    │
│ F9      ┆ 71.532847    │
│ 9E      ┆ 70.124594    │
│ EV      ┆ 65.922138    │
│ YV      ┆ 64.725458    │
└─────────┴──────────────┘


In [14]:
# Question 3.1

result = ctx.execute("""
SELECT
  f.carrier,
  a.name AS airline_name,
  f.flight,
  f.origin,
  f.dest
FROM flights f
JOIN airlines a ON f.carrier = a.carrier
LIMIT 20;
""")

print(result)

shape: (20, 5)
┌─────────┬────────────────────────┬────────┬────────┬──────┐
│ carrier ┆ airline_name           ┆ flight ┆ origin ┆ dest │
│ ---     ┆ ---                    ┆ ---    ┆ ---    ┆ ---  │
│ str     ┆ str                    ┆ i64    ┆ str    ┆ str  │
╞═════════╪════════════════════════╪════════╪════════╪══════╡
│ UA      ┆ United Air Lines Inc.  ┆ 1545   ┆ EWR    ┆ IAH  │
│ UA      ┆ United Air Lines Inc.  ┆ 1714   ┆ LGA    ┆ IAH  │
│ AA      ┆ American Airlines Inc. ┆ 1141   ┆ JFK    ┆ MIA  │
│ B6      ┆ JetBlue Airways        ┆ 725    ┆ JFK    ┆ BQN  │
│ DL      ┆ Delta Air Lines Inc.   ┆ 461    ┆ LGA    ┆ ATL  │
│ …       ┆ …                      ┆ …      ┆ …      ┆ …    │
│ B6      ┆ JetBlue Airways        ┆ 1806   ┆ JFK    ┆ BOS  │
│ UA      ┆ United Air Lines Inc.  ┆ 1187   ┆ EWR    ┆ LAS  │
│ B6      ┆ JetBlue Airways        ┆ 371    ┆ LGA    ┆ FLL  │
│ MQ      ┆ Envoy Air              ┆ 4650   ┆ LGA    ┆ ATL  │
│ B6      ┆ JetBlue Airways        ┆ 343    ┆ EWR    ┆ 

In [17]:
# Question 3.2

result = ctx.execute("""
SELECT
  f.carrier,
  AVG(2013 - p.year) AS avg_plane_age
FROM flights f
JOIN planes p ON f.tailnum = p.tailnum
WHERE p.year IS NOT NULL
GROUP BY f.carrier;
""")

print(result)

shape: (16, 2)
┌─────────┬───────────────┐
│ carrier ┆ avg_plane_age │
│ ---     ┆ ---           │
│ str     ┆ f64           │
╞═════════╪═══════════════╡
│ YV      ┆ 9.313758      │
│ AS      ┆ 3.33662       │
│ HA      ┆ 1.548387      │
│ 9E      ┆ 7.101053      │
│ US      ┆ 9.103663      │
│ …       ┆ …             │
│ B6      ┆ 6.686702      │
│ EV      ┆ 11.308998     │
│ UA      ┆ 13.207691     │
│ FL      ┆ 11.385829     │
│ DL      ┆ 16.372169     │
└─────────┴───────────────┘


In [18]:
# Question 3.3

result = ctx.execute("""
SELECT f.*
FROM flights f
JOIN weather w
  ON f.year = w.year
  AND f.month = w.month
  AND f.day = w.day
  AND f.origin = w.origin
WHERE f.dep_delay > 30
  AND (w.wind_speed > 20 OR w.precip > 0.1);
""")

print(result)

shape: (96_667, 19)
┌──────┬───────┬─────┬──────────┬───┬──────────┬──────┬────────┬─────────────────────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ distance ┆ hour ┆ minute ┆ time_hour               │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---      ┆ ---  ┆ ---    ┆ ---                     │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ i64      ┆ i64  ┆ i64    ┆ datetime[μs, UTC]       │
╞══════╪═══════╪═════╪══════════╪═══╪══════════╪══════╪════════╪═════════════════════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 811      ┆ … ┆ 544      ┆ 6    ┆ 30     ┆ 2013-01-01 11:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 826      ┆ … ┆ 1089     ┆ 7    ┆ 15     ┆ 2013-01-01 12:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 826      ┆ … ┆ 1089     ┆ 7    ┆ 15     ┆ 2013-01-01 12:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 848      ┆ … ┆ 184      ┆ 18   ┆ 35     ┆ 2013-01-01 23:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 848      ┆ … ┆ 184      ┆ 18   ┆ 35     ┆ 2013-01-01 23:00:00 UTC │
│ …    ┆ …     ┆ …   ┆ …        ┆ … ┆ …        ┆ …    ┆ …      ┆ …    

In [19]:
# Question 4.1

result = ctx.execute("""
SELECT
  p.manufacturer,
  p.model,
  COUNT(*) AS flight_count
FROM flights f
JOIN planes p ON f.tailnum = p.tailnum
GROUP BY p.manufacturer, p.model
ORDER BY flight_count DESC
LIMIT 10;
""")

print(result)

shape: (10, 3)
┌───────────────────────────────┬─────────────────┬──────────────┐
│ manufacturer                  ┆ model           ┆ flight_count │
│ ---                           ┆ ---             ┆ ---          │
│ str                           ┆ str             ┆ u32          │
╞═══════════════════════════════╪═════════════════╪══════════════╡
│ AIRBUS                        ┆ A320-232        ┆ 31278        │
│ EMBRAER                       ┆ EMB-145LR       ┆ 28027        │
│ EMBRAER                       ┆ ERJ 190-100 IGW ┆ 23716        │
│ AIRBUS INDUSTRIE              ┆ A320-232        ┆ 14553        │
│ EMBRAER                       ┆ EMB-145XR       ┆ 14051        │
│ BOEING                        ┆ 737-824         ┆ 13809        │
│ BOMBARDIER INC                ┆ CL-600-2D24     ┆ 11807        │
│ BOEING                        ┆ 737-7H4         ┆ 10389        │
│ BOEING                        ┆ 757-222         ┆ 9150         │
│ MCDONNELL DOUGLAS AIRCRAFT CO ┆ MD-88        

In [25]:
# Question 4.2 - Polars DataFrame Operations

# Join flights with airports for origin
flights_with_origin_names = flights.join(
    airports.select(['faa', 'name']),
    left_on="origin",
    right_on="faa",
    how="left"
).rename({"name": "origin_name"})

# Join the result with airports for destination
flights_with_airport_names = flights_with_origin_names.join(
    airports.select(['faa', 'name']),
    left_on="dest",
    right_on="faa",
    how="left"
).rename({"name": "dest_name"})

# Perform the aggregation
result = flights_with_airport_names.group_by(['origin', 'origin_name', 'dest', 'dest_name']).agg([
    pl.len().alias("total_flights"),
    pl.col("dep_delay").mean().alias("avg_dep_delay"),
    ((pl.col("dep_delay") > 30).sum().cast(pl.Float64) / pl.len() * 100).alias("pct_delayed_over_30")
]).sort("total_flights", descending=True).limit(10)

print(result)

shape: (10, 7)
┌────────┬────────────────┬──────┬────────────────┬───────────────┬───────────────┬────────────────┐
│ origin ┆ origin_name    ┆ dest ┆ dest_name      ┆ total_flights ┆ avg_dep_delay ┆ pct_delayed_ov │
│ ---    ┆ ---            ┆ ---  ┆ ---            ┆ ---           ┆ ---           ┆ er_30          │
│ str    ┆ str            ┆ str  ┆ str            ┆ u32           ┆ f64           ┆ ---            │
│        ┆                ┆      ┆                ┆               ┆               ┆ f64            │
╞════════╪════════════════╪══════╪════════════════╪═══════════════╪═══════════════╪════════════════╡
│ JFK    ┆ John F Kennedy ┆ LAX  ┆ Los Angeles    ┆ 11262         ┆ 8.522508      ┆ 9.829515       │
│        ┆ Intl           ┆      ┆ Intl           ┆               ┆               ┆                │
│ LGA    ┆ La Guardia     ┆ ATL  ┆ Hartsfield     ┆ 10263         ┆ 11.448621     ┆ 12.247881      │
│        ┆                ┆      ┆ Jackson        ┆               ┆         

In [31]:
# Bonus: Compare with Polars

import polars as pl

# Join flights with airports twice for origin and destination names
flights_with_airport_names = flights.join(
    airports.select(['faa', 'name']),
    left_on="origin",
    right_on="faa",
    how="left"
).rename({"name": "origin_name"})

flights_with_airport_names = flights_with_airport_names.join(
    airports.select(['faa', 'name']),
    left_on="dest",
    right_on="faa",
    how="left"
).rename({"name": "dest_name"})

# Perform the aggregation
route_stats = flights_with_airport_names.group_by(['origin', 'origin_name', 'dest', 'dest_name']).agg([
    pl.len().alias("total_flights"),
    pl.col("dep_delay").mean().alias("avg_dep_delay"),
    ((pl.col("dep_delay") > 30).sum().cast(pl.Float64) / pl.len() * 100).alias("pct_delayed_over_30")
]).sort("total_flights", descending=True).limit(10)

print(route_stats)

shape: (10, 7)
┌────────┬────────────────┬──────┬────────────────┬───────────────┬───────────────┬────────────────┐
│ origin ┆ origin_name    ┆ dest ┆ dest_name      ┆ total_flights ┆ avg_dep_delay ┆ pct_delayed_ov │
│ ---    ┆ ---            ┆ ---  ┆ ---            ┆ ---           ┆ ---           ┆ er_30          │
│ str    ┆ str            ┆ str  ┆ str            ┆ u32           ┆ f64           ┆ ---            │
│        ┆                ┆      ┆                ┆               ┆               ┆ f64            │
╞════════╪════════════════╪══════╪════════════════╪═══════════════╪═══════════════╪════════════════╡
│ JFK    ┆ John F Kennedy ┆ LAX  ┆ Los Angeles    ┆ 11262         ┆ 8.522508      ┆ 9.829515       │
│        ┆ Intl           ┆      ┆ Intl           ┆               ┆               ┆                │
│ LGA    ┆ La Guardia     ┆ ATL  ┆ Hartsfield     ┆ 10263         ┆ 11.448621     ┆ 12.247881      │
│        ┆                ┆      ┆ Jackson        ┆               ┆         